<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Gen AI Experiments](https://img.shields.io/badge/Gen%20AI%20Experiments-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://github.com/buildfastwithai/gen-ai-experiments)
[![Gen AI Experiments GitHub](https://img.shields.io/github/stars/buildfastwithai/gen-ai-experiments?style=for-the-badge&logo=github&color=gold)](http://github.com/buildfastwithai/gen-ai-experiments)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1_C4c4nQCzQNXdv1sJb24kZJZs-tpJ690?usp=sharing)


## Google Gemini Interactions API
**What You'll Learn:**
- Build with the Interactions API
- Create multi-turn conversations (stateful & stateless)
- Use multimodal capabilities
- Leverage agentic features (tools, function calling)
- Implement streaming and advanced configurations

The Interactions API is designed for building and interacting with agents, and includes support for function calling, built-in tools, structured outputs, and the Model Context Protocol (MCP).

[Learn More](https://ai.google.dev/gemini-api/docs/interactions)

## 1. Setup & Installation
Install the latest Google GenAI SDK to use the Interactions API.

In [1]:
!pip install -q google-genai

## 2. API Key and Client Setup
Initialize the GenAI client with your API key.

In [3]:
from google import genai
from google.colab import userdata

# Get your API key from Colab secrets
api_key = userdata.get("GOOGLE_API_KEY")
client = genai.Client(api_key=api_key)

print("Client initialized successfully!")

Client initialized successfully!


## 3. Basic Interactions
The simplest way to interact with the model is by providing a text prompt. The input can be a string, a list containing content objects, or a list of turns with roles and content objects.

In [4]:
# Basic text interaction
interaction = client.interactions.create(
    model="gemini-3-flash-preview",
    input="Tell me a short joke about programming."
)

print(f"Response: {interaction.outputs[-1].text}")

/tmp/ipython-input-4030357283.py:2: UserWarning: Interactions usage is experimental and may change in future versions.
  interaction = client.interactions.create(


Response: Why do programmers prefer dark mode?

Because light attracts bugs.


In [5]:
# Another basic example
interaction = client.interactions.create(
    model="gemini-3-flash-preview",
    input="What are the benefits of using Python for data science?"
)

print(f"Response: {interaction.outputs[-1].text}")

Response: Python has become the industry standard for data science, surpassing other languages like R, SAS, and MATLAB in general popularity. Its dominance is due to a unique combination of simplicity, a massive ecosystem, and versatility.

Here are the primary benefits of using Python for data science:

### 1. Simple and Readable Syntax
Python’s syntax is designed to be readable and mimics the English language. 
*   **Low Barrier to Entry:** Data scientists often come from backgrounds in math, statistics, or biology rather than computer science. Python allows them to focus on solving data problems rather than struggling with complex code structures (like pointers or memory management).
*   **Efficiency:** You can accomplish complex tasks in fewer lines of code than in C++ or Java.

### 2. An Unrivaled Ecosystem of Libraries
Python has a specialized library for every step of the data science workflow:
*   **Data Manipulation:** **Pandas** and **NumPy** are the industry standards for ha

## 4. Multi-turn Conversations
You can build multi-turn conversations in two ways:
- **Statefully**: by referencing a previous interaction
- **Statelessly**: by providing the entire conversation history

### 4.1 Stateful Conversation
Pass the `id` from the previous interaction to the `previous_interaction_id` parameter to continue a conversation. The server manages the conversation history for you.

In [6]:
# 1. First turn
interaction1 = client.interactions.create(
    model="gemini-3-flash-preview",
    input="Hi, my name is Phil."
)
print(f"Model: {interaction1.outputs[-1].text}")

# 2. Second turn (passing previous_interaction_id)
interaction2 = client.interactions.create(
    model="gemini-3-flash-preview",
    input="What is my name?",
    previous_interaction_id=interaction1.id
)
print(f"Model: {interaction2.outputs[-1].text}")

Model: Hi Phil! It's nice to meet you. How can I help you today?
Model: Your name is Phil! How can I help you today?


#### Retrieve Past Stateful Interactions
Using the interaction `id` to retrieve previous turns of the conversation.

In [7]:
# Retrieve a previous interaction using its ID
previous_interaction = client.interactions.get(interaction1.id)
print(f"Retrieved Interaction ID: {previous_interaction.id}")
print(f"Retrieved Content: {previous_interaction.outputs[-1].text}")

Retrieved Interaction ID: v1_ChdmZGxuYWVxZktyR05qckVQdklpWWdBaxIXZmRsbmFlcWZLckdOanJFUHZJaVlnQWs
Retrieved Content: Hi Phil! It's nice to meet you. How can I help you today?


### 4.2 Stateless Conversation
You can manage conversation history manually on the client side by providing the full history with each request.

In [8]:
# Manage conversation history manually
conversation_history = [
    {
        "role": "user",
        "content": "What are the three largest cities in Spain?"
    }
]

interaction1 = client.interactions.create(
    model="gemini-3-flash-preview",
    input=conversation_history
)
print(f"Model: {interaction1.outputs[-1].text}")

# Add model response and new user message to history
conversation_history.append({"role": "model", "content": interaction1.outputs})
conversation_history.append({
    "role": "user",
    "content": "What is the most famous landmark in the second one?"
})

interaction2 = client.interactions.create(
    model="gemini-3-flash-preview",
    input=conversation_history
)
print(f"Model: {interaction2.outputs[-1].text}")

Model: The three largest cities in Spain by population are:

1.  **Madrid** (approx. 3.3 million)
2.  **Barcelona** (approx. 1.6 million)
3.  **Valencia** (approx. 800,000)
Model: The most famous landmark in **Barcelona** is the **Sagrada Família**, the massive unfinished minor basilica designed by the architect Antoni Gaudí.


## 6. Agentic Capabilities
The Interactions API is designed for building and interacting with agents, and includes support for function calling, built-in tools, structured outputs, and the Model Context Protocol (MCP).

### 6.1 Deep Research Agent
Use specialized agents like `deep-research-pro-preview-12-2025` for complex research tasks.

In [11]:
import time

# 1. Start the Deep Research Agent
initial_interaction = client.interactions.create(
    input="Research the history of the Google TPUs with a focus on 2025 and 2026.",
    agent="deep-research-pro-preview-12-2025",
    background=True
)

print(f"Research started. Interaction ID: {initial_interaction.id}")

# 2. Poll for results
while True:
    interaction = client.interactions.get(initial_interaction.id)
    print(f"Status: {interaction.status}")

    if interaction.status == "completed":
        print("\nFinal Report:\n", interaction.outputs[-1].text)
        break
    elif interaction.status in ["failed", "cancelled"]:
        print(f"Failed with status: {interaction.status}")
        break

    time.sleep(10)

Research started. Interaction ID: v1_ChdwOWxuYWJMZlBPM3FqckVQdFoyczhRaxIXcDlsbmFiTGZQTzNxanJFUHRaMnM4UWs
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: completed

Final Report:
 # The Evolution of Google Tensor Processing Units: A Historical Analysis with Emphasis on the 2025-2026 Paradigm Shift

#

### 6.2 Function Calling (Tools)
Define custom tools and use function calling within the Interactions API.

In [12]:
# 1. Define the tool
def get_weather(location: str):
    """Gets the weather for a given location."""
    return f"The weather in {location} is sunny."

weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Gets the weather for a given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {"type": "string", "description": "The city and state, e.g. San Francisco, CA"}
        },
        "required": ["location"]
    }
}

# 2. Send the request with tools
interaction = client.interactions.create(
    model="gemini-3-flash-preview",
    input="What is the weather in Paris?",
    tools=[weather_tool]
)

# 3. Handle the tool call
for output in interaction.outputs:
    if output.type == "function_call":
        print(f"Tool Call: {output.name}({output.arguments})")

        # Execute tool
        result = get_weather(**output.arguments)

        # Send result back
        interaction = client.interactions.create(
            model="gemini-3-flash-preview",
            previous_interaction_id=interaction.id,
            input=[{
                "type": "function_result",
                "name": output.name,
                "call_id": output.id,
                "result": result
            }]
        )
        print(f"Response: {interaction.outputs[-1].text}")

Tool Call: get_weather({'location': 'Paris'})
Response: The weather in Paris is currently sunny.


### 6.3 Structured Output (JSON Schema)
Enforce a specific JSON output by providing a JSON schema in the `response_format` parameter. This is useful for tasks like moderation, classification, or data extraction.

In [13]:
from pydantic import BaseModel, Field
from typing import Literal, Union

class SpamDetails(BaseModel):
    reason: str = Field(description="The reason why the content is considered spam.")
    spam_type: Literal["phishing", "scam", "unsolicited promotion", "other"]

class NotSpamDetails(BaseModel):
    summary: str = Field(description="A brief summary of the content.")
    is_safe: bool = Field(description="Whether the content is safe for all audiences.")

class ModerationResult(BaseModel):
    decision: Union[SpamDetails, NotSpamDetails]

# Create interaction with structured output
interaction = client.interactions.create(
    model="gemini-3-flash-preview",
    input="Moderate the following content: 'Congratulations! You've won a free cruise. Click here to claim your prize: www.definitely-not-a-scam.com'",
    response_format=ModerationResult.model_json_schema(),
)

# Parse the output
parsed_output = ModerationResult.model_validate_json(interaction.outputs[-1].text)
print(f"Moderation Result: {parsed_output}")

Moderation Result: decision=SpamDetails(reason='The content uses a generic prize claim and a suspicious URL to lure users into a potential fraud scheme.', spam_type='scam')


## 7. Advanced Features

### 7.1 Streaming
Receive responses incrementally as they are generated.

In [14]:
# Stream responses
stream = client.interactions.create(
    model="gemini-3-flash-preview",
    input="Explain quantum entanglement in simple terms.",
    stream=True
)

for chunk in stream:
    if chunk.event_type == "content.delta":
        if chunk.delta.type == "text":
            print(chunk.delta.text, end="", flush=True)
        elif chunk.delta.type == "thought":
            print(chunk.delta.thought, end="", flush=True)
    elif chunk.event_type == "interaction.complete":
        print(f"\n\n--- Stream Finished ---")
        print(f"Total Tokens: {chunk.interaction.usage.total_tokens}")

Imagine you have two **magic coins**. These coins are special because they are "entangled."

Here is how they work:

### 1. The Connection
You keep one coin and give the other to a friend who travels to the other side of the galaxy. 

Normally, if you flip your coin, it has a 50/50 chance of being Heads or Tails. Your friend's coin would also have a 50/50 chance. What you get shouldn't affect what they get.

But because these are **entangled**, the moment you flip your coin and see it is "Heads," your friend’s coin **instantly** becomes "Heads" too—even though they are billions of miles away.

### 2. The "Spooky" Part
The weirdest part is that before you look at your coin, it isn't Heads or Tails. It is actually a blurry mix of both at the same time (scientists call this *superposition*). 

The two coins act like a single unit. They don't send a signal to each other through the air; they are simply linked by a fundamental law of the universe. Albert Einstein found this so strange that 

### 7.2 Configuration
Customize the model's behavior with `generation_config`.

In [15]:
# Customize generation with config
interaction = client.interactions.create(
    model="gemini-3-flash-preview",
    input="Tell me a story about a brave knight.",
    generation_config={
        "temperature": 0.7,
        "max_output_tokens": 500,
        "thinking_level": "low",
    }
)

print(interaction.outputs[-1].text)

In the kingdom of Aethelgard, bravery was usually measured by the size of one’s


### 7.3 System Instructions
Provide system-level instructions to guide model behavior.

In [16]:
# Use system instructions
interaction = client.interactions.create(
    model="gemini-3-flash-preview",
    input="What is the capital of France?",
    system_instruction="You are a poetic assistant. Answer only in rhymes."
)

print(interaction.outputs[-1].text)

The city of light, where romance is grand,
Paris is the capital of that fair land.


## 8. Supported Models & Agents
The Interactions API supports the following models and agents:

**Models:**
- `gemini-2.5-pro`
- `gemini-2.5-flash`
- `gemini-2.5-flash-lite`
- `gemini-3-pro-preview`
- `gemini-3-flash-preview`

**Agents:**
- `deep-research-pro-preview-12-2025`

## 9. How the Interactions API Works

The Interactions API is designed around a central resource: the **Interaction**.

An Interaction represents a complete turn in a conversation or task. It acts as a session record, containing the entire history of an interaction, including:
- All user inputs
- Model thoughts
- Tool calls
- Tool results
- Final model outputs

When you make a call to `interactions.create`, you are creating a new Interaction resource.

Optionally, you can use the `id` of this resource in a subsequent call using the `previous_interaction_id` parameter to continue the conversation. The server uses this ID to retrieve the full context, saving you from having to resend the entire chat history.

This server-side state management is optional; you can also operate in stateless mode by sending the full conversation history in each request.

## 10. Comparison: Interactions API vs generate_content

Here's a quick comparison to understand when to use each:

In [17]:
# Traditional generate_content approach
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Explain what an API is in one sentence."
)
print(f"generate_content: {response.text}")

# Interactions API approach
interaction = client.interactions.create(
    model="gemini-3-flash-preview",
    input="Explain what an API is in one sentence."
)
print(f"Interactions API: {interaction.outputs[-1].text}")

generate_content: An API (Application Programming Interface) is a set of definitions and protocols that allows different software applications to communicate and interact with each other.
Interactions API: An API (Application Programming Interface) is a set of rules that allows different software applications to communicate and share data with each other.


## Conclusion

In this notebook, we covered:

1. **Basic Interactions** - Simple text prompts and responses
2. **Multi-turn Conversations** - Both stateful (server-managed) and stateless (client-managed)
3. **Multimodal Capabilities** - Image understanding with base64 encoding
4. **Agentic Capabilities** - Deep Research Agent, Function Calling, and Structured Outputs
5. **Advanced Features** - Streaming, Configuration, and System Instructions

The Interactions API provides a powerful foundation for building sophisticated AI agents and applications!

[Learn more in the official documentation](https://ai.google.dev/gemini-api/docs/interactions)